In [ ]:
import numpy as np

def V(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def gradient(x, y):
    gx = 4*x*(x**2 + y - 11) + 2*(x + y**2 - 7)
    gy = 2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)
    return np.array([gx, gy])

def numerical_gradient(f, x, y, h=1e-6):
    df_dx = (f(x + h, y) - f(x - h, y)) / (2*h)
    df_dy = (f(x, y + h) - f(x, y - h)) / (2*h)
    return np.array([df_dx, df_dy])

# ✅ Gradient check (use a non-stationary point)
x_check, y_check = 1.0, 1.0
g_analytical = gradient(x_check, y_check)
g_numerical = numerical_gradient(V, x_check, y_check)
num_norm = np.linalg.norm(g_numerical)
gradient_check_error = (
    np.linalg.norm(g_analytical - g_numerical) / num_norm if num_norm > 1e-12 else 0.0
)

# Gradient Descent
eta = 1e-3
max_iter = 50000
tol = 1e-6
x, y = 0.0, 0.0
trajectory = [(x, y)]

for i in range(max_iter):
    g = gradient(x, y)
    if np.linalg.norm(g) < tol:
        break
    x -= eta * g[0]
    y -= eta * g[1]
    trajectory.append((x, y))

final_point = np.array([x, y])
final_value = V(x, y)
iterations = len(trajectory)

# Hessian analytic
def hessian(x, y):
    hxx = 12*x**2 + 4*y - 42
    hxy = 4*x + 4*y
    hyy = 12*y**2 + 4*x - 26
    return np.array([[hxx, hxy], [hxy, hyy]])

H_analytic = hessian(x, y)

# Hessian numerical via finite differences of gradient
def numerical_hessian(grad_func, x, y, h=1e-5):
    g_xph = grad_func(x + h, y)
    g_xmh = grad_func(x - h, y)
    g_yph = grad_func(x, y + h)
    g_ymh = grad_func(x, y - h)
    d2x = (g_xph[0] - g_xmh[0]) / (2*h)
    d2xy = (g_yph[0] - g_ymh[0]) / (2*h)
    d2yx = (g_xph[1] - g_xmh[1]) / (2*h)
    d2y = (g_yph[1] - g_ymh[1]) / (2*h)
    return np.array([[d2x, d2xy], [d2yx, d2y]])

H_numerical = numerical_hessian(gradient, x, y)
hessian_valid = np.allclose(H_analytic, H_numerical, atol=1e-5)

# Eigenvalue and conditioning analysis
eigvals, eigvecs = np.linalg.eig(H_analytic)
lambda_min = float(np.min(eigvals))
lambda_max = float(np.max(eigvals))
condition_number = float(abs(lambda_max / lambda_min))
is_positive_definite = bool(np.all(eigvals > 0))

# Optimization report
optimization_report = {
    "converged": bool(np.linalg.norm(gradient(x, y)) < tol),
    "iterations": iterations,
    "final_point": final_point.tolist(),
    "final_value": float(final_value),
    "gradient_check_error": float(gradient_check_error),
    "hessian_valid": bool(hessian_valid),
    "is_positive_definite": is_positive_definite,
    "condition_number": float(condition_number)
}
